In [4]:
#Current goal: Save all the pairs of (pre,post) here for LIC and run the codes
import os

low_income = ['congo', 'haiti']
pre_event_dir = r"C:\Users\sweta\Downloads\pre-event\pre-event_wo_ukraine_myanmar_mexico"
post_event_dir = r"C:\Users\sweta\Downloads\post-event\post-event"

all_pre_files = os.listdir(pre_event_dir)
filtered_pre_files = [f for f in all_pre_files if any(country.lower() in f.lower() for country in low_income)]
expected_post_files = [f.replace('_pre_disaster', '_post_disaster') for f in filtered_pre_files]
all_post_files = os.listdir(post_event_dir)
existing_post_files = [f for f in expected_post_files if f in all_post_files]

pairs = []
for pre_file, post_file in zip(filtered_pre_files, expected_post_files):
    if post_file in all_post_files:
        pairs.append((os.path.join(pre_event_dir, pre_file), os.path.join(post_event_dir, post_file)))
#----Output---
print(f"Number of filtered pre-event files matching LIC: {len(filtered_pre_files)}")
print(f"Number of expected post-event files generated: {len(expected_post_files)}")
print(f"Number of existing post-event files found: {len(existing_post_files)}")
print(f"Number of matched pre/post pairs: {len(pairs)}")

if len(filtered_pre_files) == len(existing_post_files) == len(pairs):
    print("Perfect 1:1 correspondence found between pre and post event files.")
else:
    print("Mismatch detected between pre and post event files.")

Number of filtered pre-event files matching LIC: 196
Number of expected post-event files generated: 196
Number of existing post-event files found: 196
Number of matched pre/post pairs: 196
Perfect 1:1 correspondence found between pre and post event files.


In [8]:
#just need for sanity
#all imports
import os
import random
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.nn.utils import spectral_norm

#----
#Configuration
#----
class Config:
    seed = 42
    img_size = 256
    batch_size = 16
    epochs = 10  # 10-epoch training
    lr = 2e-4
    betas = (0.5, 0.999)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Base loss weights (dynamically adjusted)
    lambda_cls = 1
    lambda_mask = 10
    lambda_gp = 10
    lambda_cycle = 10
    
    # Paths - updated for your local Windows system
    data_root = r"C:\Users\sweta\.cache\kagglehub\datasets\qianlanzz\xbd-dataset\versions\1\xbd"
    save_dir = r"C:\Users\sweta\anaconda_projects\non-trivial\saved_models"
    samples_dir = r"C:\Users\sweta\anaconda_projects\non-trivial\samples"
    plots_dir = r"C:\Users\sweta\anaconda_projects\non-trivial\plots"
    
    disaster_types = [
        'volcano', 'fire', 'tornado', 'tsunami',
        'flooding', 'earthquake', 'hurricane'
    ]

config = Config()

# Create directories if they don't exist
os.makedirs(config.save_dir, exist_ok=True)
os.makedirs(config.samples_dir, exist_ok=True)
os.makedirs(config.plots_dir, exist_ok=True)

# -------------------------
# Dataset Implementation
# -------------------------
class XBDDataset(Dataset):
    def __init__(self, split_names=("train", "tier1", "tier3")):
        self.pairs = []
        for split in split_names:
            split_path = os.path.join(config.data_root, split)
            images_dir = os.path.join(split_path, "images")
            labels_dir = os.path.join(split_path, "labels")
            
            pre_images = glob.glob(os.path.join(images_dir, "*_pre_disaster.png"))
            for pre_path in pre_images:
                base = os.path.basename(pre_path).replace("_pre_disaster.png", "")
                post_path = os.path.join(images_dir, f"{base}_post_disaster.png")
                label_path = os.path.join(labels_dir, f"{base}_post_disaster.json")
                
                if os.path.exists(post_path):
                    self.pairs.append({
                        "pre": pre_path,
                        "post": post_path,
                        "label": label_path if os.path.exists(label_path) else None
                    })

        self.transform = transforms.Compose([
            transforms.Resize(config.img_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def __len__(self):
        return len(self.pairs)

    def parse_disaster(self, label_path):
        try:
            with open(label_path) as f:
                data = json.load(f)
            disaster = data['metadata']['disaster_type']
            return torch.tensor(config.disaster_types.index(disaster), dtype=torch.long)
        except:
            return torch.tensor(0, dtype=torch.long)

    def __getitem__(self, idx):
        pair = self.pairs[idx]
        pre_img = self.transform(Image.open(pair["pre"]).convert('RGB'))
        post_img = self.transform(Image.open(pair["post"]).convert('RGB'))
        disaster_label = self.parse_disaster(pair["label"])
        
        with torch.no_grad():
            diff = torch.abs(post_img - pre_img).mean(dim=0, keepdim=True)
            mask = (diff > 0.1).float()
        
        return {
            'pre': pre_img,
            'post': post_img,
            'disaster': disaster_label,
            'mask': mask
        }

# -------------------------
# Generator Network
# -------------------------
class DisasterGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = nn.Sequential(
            spectral_norm(nn.Conv2d(4, 64, 4, 2, 1)),
            nn.LeakyReLU(0.2)
        )
        self.enc2 = nn.Sequential(
            spectral_norm(nn.Conv2d(64, 128, 4, 2, 1)),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2)
        )
        self.enc3 = nn.Sequential(
            spectral_norm(nn.Conv2d(128, 256, 4, 2, 1)),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2)
        )
        self.enc4 = nn.Sequential(
            spectral_norm(nn.Conv2d(256, 512, 4, 2, 1)),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2)
        )
        
        # Decoders
        self.dec_img = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.InstanceNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1),
            nn.Tanh()
        )
        
        self.dec_mask = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.InstanceNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, 4, 2, 1),
            nn.Sigmoid()
        )

    def add_disaster_channel(self, x, disaster):
        batch_size, _, h, w = x.size()
        disaster_map = disaster.view(-1, 1, 1, 1).expand(-1, -1, h, w).float() / len(config.disaster_types)
        return torch.cat([x, disaster_map], dim=1)

    def forward(self, x, disaster):
        x = self.add_disaster_channel(x, disaster)
        
        # Encoding
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        
        # Decoding
        img = self.dec_img(e4)
        mask = self.dec_mask(e4)
        return img, mask

# -------------------------
# Discriminator Network
# -------------------------
class DisasterDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            spectral_norm(nn.Conv2d(3, 64, 4, 2, 1)),
            nn.LeakyReLU(0.2),
            spectral_norm(nn.Conv2d(64, 128, 4, 2, 1)),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2),
            spectral_norm(nn.Conv2d(128, 256, 4, 2, 1)),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2),
            spectral_norm(nn.Conv2d(256, 512, 4, 2, 1)),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2),
        )
        
        self.src = spectral_norm(nn.Conv2d(512, 1, 4, 1, 1))
        self.cls = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            spectral_norm(nn.Conv2d(512, len(config.disaster_types), 1)),
            nn.Flatten()
        )

    def forward(self, x):
        features = self.main(x)
        return self.src(features), self.cls(features)

import torch
from torchvision import transforms as T
from PIL import Image
import matplotlib.pyplot as plt

# Initialize generator model
G = DisasterGenerator().to(config.device)

# Load trained weights
G.load_state_dict(torch.load(r"C:\Users\sweta\anaconda_projects\non-trivial\saved_models\G_final.pth", map_location=config.device))

# Set to evaluation mode
G.eval()

print("Loaded Generator Model")

from torchvision import transforms as T

transform_rgb = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)
])


Loaded Generator Model


C:\Users\sweta\AppData\Local\Temp\ipykernel_8872\1320358088.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  G.load_state_dict(torch.load(r"C:\Users\sweta\anaconda_proj

In [27]:
import torch
from torchvision import transforms as T
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import os
import json

# Model setup
device = config.device
G.eval()

disaster_types = ['earthquake', 'flood', 'hurricane', 'fire', 'tsunami', 'volcano']

# Disaster label inference
def infer_disaster_type(filename):
    for d in disaster_types:
        if d in filename.lower():
            return d
    return None

transform_rgb = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)
])

transform_gray = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize([0.5], [0.5])
])

def tensor_to_pil(tensor):
    """
    Convert a normalized tensor (C,H,W) with values in [-1,1] to a PIL RGB image.
    """
    tensor = tensor.squeeze(0).cpu()  # Remove batch dim, move to CPU
    tensor = (tensor * 0.5) + 0.5     # Denormalize from [-1,1] to [0,1]
    tensor = torch.clamp(tensor, 0, 1)
    np_img = tensor.permute(1, 2, 0).numpy() * 255
    np_img = np_img.astype(np.uint8)
    return Image.fromarray(np_img)


# SAR mask function
def SAR_damage_mask(pre_optical_img, post_sar_img, threshold=0.1):
    def optical_to_sar_like(img):
        img = img.convert('L')
        img = ImageOps.autocontrast(img, cutoff=2)
        return img

    pre_sar_like = optical_to_sar_like(pre_optical_img)
    post_sar = post_sar_img.convert('L')

    pre_tensor = transform_gray(pre_sar_like).to(device)
    post_tensor = transform_gray(post_sar).to(device)

    with torch.no_grad():
        diff = torch.abs(post_tensor - pre_tensor).unsqueeze(0)
        diff = F.avg_pool2d(diff, kernel_size=3, stride=1, padding=1)
        img_std = diff.std()
        adaptive_threshold = threshold + (0.1 * img_std)
        mask = (diff > adaptive_threshold).float()
        mask = F.max_pool2d(mask, kernel_size=3, stride=1, padding=1)
        mask = F.avg_pool2d(mask, kernel_size=3, stride=1, padding=1)

    return mask.squeeze(0)

# Metric calculation
def compute_mask_metrics(pred_mask, true_mask):
    pred = pred_mask.flatten().cpu().numpy().astype(int)
    true = true_mask.flatten().cpu().numpy().astype(int)
    return {
        'IoU': jaccard_score(true, pred),
        'Dice': f1_score(true, pred),
        'F1': f1_score(true, pred),
        'Precision': precision_score(true, pred),
        'Recall': recall_score(true, pred)
    }

# 95% Confidence Interval function
def compute_ci(arr):
    mean = np.mean(arr)
    std_err = np.std(arr) / np.sqrt(len(arr))
    return (mean - 1.96 * std_err, mean + 1.96 * std_err)

# Get all pre-post pairs
low_income = ['congo', 'haiti']
pre_event_dir = r"C:\Users\sweta\Downloads\pre-event\pre-event_wo_ukraine_myanmar_mexico"
post_event_dir = r"C:\Users\sweta\Downloads\post-event\post-event"
all_pre_files = os.listdir(pre_event_dir)
filtered_pre_files = [f for f in all_pre_files if any(country.lower() in f.lower() for country in low_income)]
expected_post_files = [f.replace('_pre_disaster', '_post_disaster') for f in filtered_pre_files]
all_post_files = os.listdir(post_event_dir)
pairs = [
    (os.path.join(pre_event_dir, pre), os.path.join(post_event_dir, post))
    for pre, post in zip(filtered_pre_files, expected_post_files)
    if post in all_post_files
]

# Output directories setup
output_dir = r"C:\Users\sweta\anaconda_projects\non-trivial\evaluation_bias\lic_results"
os.makedirs(output_dir, exist_ok=True)

disaster_output_dirs = {}
for d in disaster_types:
    dir_path = os.path.join(output_dir, d)
    os.makedirs(dir_path, exist_ok=True)
    disaster_output_dirs[d] = dir_path

# Evaluation loop
all_pair_metrics = []
disaster_type_metrics = {d: [] for d in disaster_types}

for pair_idx, (pre_path, post_path) in enumerate(pairs):
    print(f"\n=== Pair {pair_idx+1}/{len(pairs)} ===")
    pre_img_full = Image.open(pre_path).convert('RGB')
    post_img_full = Image.open(post_path).convert('RGB')

    tile_size = 256
    width, height = pre_img_full.size
    tiles = []

    for top in range(0, height, tile_size):
        for left in range(0, width, tile_size):
            box = (left, top, left + tile_size, top + tile_size)
            if box[2] <= width and box[3] <= height:
                pre_tile = pre_img_full.crop(box)
                post_tile = post_img_full.crop(box)
                tiles.append((pre_tile, post_tile))

    tile_metrics = []

    for idx, (pre_img, post_img) in enumerate(tiles):
        input_tensor = transform_rgb(pre_img).unsqueeze(0).to(device)
        post_resized = post_img.resize((256, 256))

        inferred_type = infer_disaster_type(pre_path)
        if inferred_type is None:
            print(f"Unknown disaster type for file: {pre_path}")
            continue

        disaster_idx = disaster_types.index(inferred_type)
        disaster_label = torch.tensor([[disaster_idx]], device=device)

        with torch.no_grad():
            fake_post, pred_mask = G(input_tensor, disaster_label)
        actual_mask = SAR_damage_mask(pre_img, post_resized, threshold=0.2)
        fake_post_img = tensor_to_pil(fake_post)
        pred_mask = SAR_damage_mask(pre_img, fake_post_img, threshold=0.2)
        metrics = compute_mask_metrics(pred_mask[0], actual_mask[0])
        tile_metrics.append(metrics)

        # Save visualization every 10th tile
        if idx % 10 == 0:
            vis_dir = os.path.join(disaster_output_dirs[inferred_type], "visualizations")
            os.makedirs(vis_dir, exist_ok=True)

            fig, axs = plt.subplots(1, 4, figsize=(16, 4))

            axs[0].imshow(pre_img)
            axs[0].set_title("Pre-disaster")
            axs[0].axis('off')

            axs[1].imshow(post_img)
            axs[1].set_title("Post-disaster (True)")
            axs[1].axis('off')

            axs[2].imshow(fake_post_img)
            axs[2].set_title("Fake Post (Generated)")
            axs[2].axis('off')

            axs[3].imshow(pred_mask[0].cpu(), cmap='gray')
            axs[3].set_title("Predicted Damage Mask")
            axs[3].axis('off')

            plt.suptitle(f"Pair {pair_idx+1}, Tile {idx} ({inferred_type})")
            save_path = os.path.join(vis_dir, f"pair{pair_idx+1}_tile{idx}.png")
            plt.savefig(save_path)
            plt.close(fig)

    mean_metrics = {
        'IoU': np.mean([m['IoU'] for m in tile_metrics]),
        'Dice': np.mean([m['Dice'] for m in tile_metrics]),
        'F1': np.mean([m['F1'] for m in tile_metrics]),
        'Precision': np.mean([m['Precision'] for m in tile_metrics]),
        'Recall': np.mean([m['Recall'] for m in tile_metrics]),
    }
    all_pair_metrics.append({
        'pair': (os.path.basename(pre_path), os.path.basename(post_path)),
        'metrics': mean_metrics
    })
    disaster_type_metrics[inferred_type].append(mean_metrics)

    print("Mean metrics for this pair:")
    for k, v in mean_metrics.items():
        print(f"  {k}: {v:.3f}")

# Overall stats
print("\n=== Overall Summary Across All Pairs ===")
all_iou = [m['metrics']['IoU'] for m in all_pair_metrics]
all_dice = [m['metrics']['Dice'] for m in all_pair_metrics]
all_f1 = [m['metrics']['F1'] for m in all_pair_metrics]
all_prec = [m['metrics']['Precision'] for m in all_pair_metrics]
all_rec = [m['metrics']['Recall'] for m in all_pair_metrics]

print(f"Average IoU: {np.mean(all_iou):.3f}")
print(f"Median IoU: {np.median(all_iou):.3f}")
print(f"Average Dice: {np.mean(all_dice):.3f}")
print(f"Median Dice: {np.median(all_dice):.3f}")
print(f"Average F1: {np.mean(all_f1):.3f}")
print(f"Median F1: {np.median(all_f1):.3f}")
print(f"Average Precision: {np.mean(all_prec):.3f}")
print(f"Median Precision: {np.median(all_prec):.3f}")
print(f"Average Recall: {np.mean(all_rec):.3f}")
print(f"Median Recall: {np.median(all_rec):.3f}")

# Save global results
with open(os.path.join(output_dir, "per_pair_metrics.json"), "w") as f:
    json.dump(all_pair_metrics, f, indent=2)

summary_stats = {
    'Average IoU': float(np.mean(all_iou)),
    'Median IoU': float(np.median(all_iou)),
    '95% CI IoU': list(compute_ci(all_iou)),

    'Average Dice': float(np.mean(all_dice)),
    'Median Dice': float(np.median(all_dice)),
    '95% CI Dice': list(compute_ci(all_dice)),

    'Average F1': float(np.mean(all_f1)),
    'Median F1': float(np.median(all_f1)),
    '95% CI F1': list(compute_ci(all_f1)),

    'Average Precision': float(np.mean(all_prec)),
    'Median Precision': float(np.median(all_prec)),
    '95% CI Precision': list(compute_ci(all_prec)),

    'Average Recall': float(np.mean(all_rec)),
    'Median Recall': float(np.median(all_rec)),
    '95% CI Recall': list(compute_ci(all_rec)),
}

with open(os.path.join(output_dir, "summary_stats.json"), "w") as f:
    json.dump(summary_stats, f, indent=2)

# Save results per disaster type
for d in disaster_types:
    metrics_list = disaster_type_metrics[d]
    if not metrics_list:
        continue  # skip if no data for this disaster

    iou_vals = [m['IoU'] for m in metrics_list]
    dice_vals = [m['Dice'] for m in metrics_list]
    f1_vals = [m['F1'] for m in metrics_list]
    prec_vals = [m['Precision'] for m in metrics_list]
    rec_vals = [m['Recall'] for m in metrics_list]

    summary = {
        'Average IoU': float(np.mean(iou_vals)),
        'Median IoU': float(np.median(iou_vals)),
        '95% CI IoU': list(compute_ci(iou_vals)),

        'Average Dice': float(np.mean(dice_vals)),
        'Median Dice': float(np.median(dice_vals)),
        '95% CI Dice': list(compute_ci(dice_vals)),

        'Average F1': float(np.mean(f1_vals)),
        'Median F1': float(np.median(f1_vals)),
        '95% CI F1': list(compute_ci(f1_vals)),

        'Average Precision': float(np.mean(prec_vals)),
        'Median Precision': float(np.median(prec_vals)),
        '95% CI Precision': list(compute_ci(prec_vals)),

        'Average Recall': float(np.mean(rec_vals)),
        'Median Recall': float(np.median(rec_vals)),
        '95% CI Recall': list(compute_ci(rec_vals)),
    }

    per_pair_path = os.path.join(disaster_output_dirs[d], f"per_pair_metrics_{d}.json")
    summary_path = os.path.join(disaster_output_dirs[d], f"summary_stats_{d}.json")

    disaster_pairs = [p for p in all_pair_metrics if infer_disaster_type(p['pair'][0]) == d]

    with open(per_pair_path, 'w') as f:
        json.dump(disaster_pairs, f, indent=2)
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)

    print(f"Saved results for disaster '{d}' to {disaster_output_dirs[d]}")

print(f"\nSaved all results to: {output_dir}")



=== Pair 1/196 ===


C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: Unde

Mean metrics for this pair:
  IoU: 0.585
  Dice: 0.603
  F1: 0.603
  Precision: 0.612
  Recall: 0.596

=== Pair 2/196 ===
Mean metrics for this pair:
  IoU: 0.889
  Dice: 0.935
  F1: 0.935
  Precision: 0.973
  Recall: 0.907

=== Pair 3/196 ===
Mean metrics for this pair:
  IoU: 0.964
  Dice: 0.981
  F1: 0.981
  Precision: 0.989
  Recall: 0.974

=== Pair 4/196 ===
Mean metrics for this pair:
  IoU: 0.951
  Dice: 0.974
  F1: 0.974
  Precision: 0.978
  Recall: 0.972

=== Pair 5/196 ===
Mean metrics for this pair:
  IoU: 0.966
  Dice: 0.983
  F1: 0.983
  Precision: 0.992
  Recall: 0.974

=== Pair 6/196 ===
Mean metrics for this pair:
  IoU: 0.944
  Dice: 0.970
  F1: 0.970
  Precision: 0.983
  Recall: 0.958

=== Pair 7/196 ===
Mean metrics for this pair:
  IoU: 0.931
  Dice: 0.962
  F1: 0.962
  Precision: 0.986
  Recall: 0.943

=== Pair 8/196 ===
Mean metrics for this pair:
  IoU: 0.930
  Dice: 0.963
  F1: 0.963
  Precision: 0.979
  Recall: 0.949

=== Pair 9/196 ===


C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mean metrics for this pair:
  IoU: 0.689
  Dice: 0.754
  F1: 0.754
  Precision: 0.772
  Recall: 0.839

=== Pair 10/196 ===


...

C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: Unde

Mean metrics for this pair:
  IoU: 0.546
  Dice: 0.603
  F1: 0.603
  Precision: 0.589
  Recall: 0.630

=== Pair 196/196 ===


C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: Unde

Mean metrics for this pair:
  IoU: 0.362
  Dice: 0.414
  F1: 0.414
  Precision: 0.405
  Recall: 0.430

=== Overall Summary Across All Pairs ===
Average IoU: 0.633
Median IoU: 0.689
Average Dice: 0.730
Median Dice: 0.797
Average F1: 0.730
Median F1: 0.797
Average Precision: 0.835
Median Precision: 0.904
Average Recall: 0.678
Median Recall: 0.749
Saved results for disaster 'earthquake' to C:\Users\sweta\anaconda_projects\non-trivial\evaluation_bias\lic_results\earthquake
Saved results for disaster 'volcano' to C:\Users\sweta\anaconda_projects\non-trivial\evaluation_bias\lic_results\volcano

Saved all results to: C:\Users\sweta\anaconda_projects\non-trivial\evaluation_bias\lic_results


C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
print("Pred mask shape:", pred_mask.shape)
print("Actual mask shape:", actual_mask.shape)


Pred mask shape: torch.Size([1, 256, 256])
Actual mask shape: torch.Size([1, 256, 256])


In [32]:
#multi class
import torch
from torchvision import transforms as T
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import os
import json

# Model setup
device = config.device
G.eval()

disaster_types = ['earthquake', 'flood', 'hurricane', 'fire', 'tsunami', 'volcano']

# Disaster label inference
def infer_disaster_type(filename):
    for d in disaster_types:
        if d in filename.lower():
            return d
    return None

transform_rgb = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)
])

transform_gray = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize([0.5], [0.5])
])

def tensor_to_pil(tensor):
    """
    Convert a normalized tensor (C,H,W) with values in [-1,1] to a PIL RGB image.
    """
    tensor = tensor.squeeze(0).cpu()  # Remove batch dim, move to CPU
    tensor = (tensor * 0.5) + 0.5     # Denormalize from [-1,1] to [0,1]
    tensor = torch.clamp(tensor, 0, 1)
    np_img = tensor.permute(1, 2, 0).numpy() * 255
    np_img = np_img.astype(np.uint8)
    return Image.fromarray(np_img)
'''
# Multiclass SAR damage mask
def SAR_damage_mask_multiclass_merged(pre_optical_img, post_sar_img):
    def optical_to_sar_like(img):
        img = img.convert('L')
        img = ImageOps.autocontrast(img, cutoff=2)
        return img

    pre_sar_like = optical_to_sar_like(pre_optical_img)
    post_sar = post_sar_img.convert('L')

    pre_tensor = transform_gray(pre_sar_like).to(device).squeeze(0)
    post_tensor = transform_gray(post_sar).to(device).squeeze(0)

    with torch.no_grad():
        diff = torch.abs(post_tensor - pre_tensor)  # shape (H, W)
        diff_smoothed = F.avg_pool2d(diff.unsqueeze(0).unsqueeze(0), kernel_size=3, stride=1, padding=1).squeeze()

        # Normalize diff to [0,1]
        diff_min, diff_max = diff_smoothed.min(), diff_smoothed.max()
        diff_norm = (diff_smoothed - diff_min) / (diff_max - diff_min + 1e-8)

        # Thresholds to classify damage severity (excluding background/no damage separation)
        thresholds = [0.1, 0.25, 0.5]  # tune these empirically

        mask = torch.zeros_like(diff_norm, dtype=torch.long)  # 0 = no damage (including background)

        mask = torch.where((diff_norm >= thresholds[0]) & (diff_norm < thresholds[1]), 1, mask)  # Minor damage
        mask = torch.where((diff_norm >= thresholds[1]) & (diff_norm < thresholds[2]), 2, mask)  # Major damage
        mask = torch.where(diff_norm >= thresholds[2], 3, mask)  # Destroyed

    return mask.cpu()  # (H, W) tensor with int labels 0-3
    '''
import numpy as np
from scipy.ndimage import binary_opening, binary_closing

def clean_multiclass_mask(mask_np, min_region_size=20):
    cleaned_mask = np.zeros_like(mask_np)
    for cls in range(1, 4):  # Skip 0 (no damage)
        binary = (mask_np == cls)
        binary = binary_opening(binary, structure=np.ones((3,3)))
        binary = binary_closing(binary, structure=np.ones((3,3)))
        cleaned_mask[binary] = cls
    return cleaned_mask

def SAR_damage_mask_multiclass_merged(pre_optical_img, post_sar_img):
    def optical_to_sar_like(img):
        img = img.convert('L')
        img = ImageOps.autocontrast(img, cutoff=2)
        return img

    pre_sar_like = optical_to_sar_like(pre_optical_img)
    post_sar = post_sar_img.convert('L')

    pre_tensor = transform_gray(pre_sar_like).to(device).squeeze(0)
    post_tensor = transform_gray(post_sar).to(device).squeeze(0)

    with torch.no_grad():
        diff = torch.abs(post_tensor - pre_tensor)  # shape (H, W)
        diff_smoothed = F.avg_pool2d(diff.unsqueeze(0).unsqueeze(0), kernel_size=5, stride=1, padding=2).squeeze()

        # Normalize diff to [0,1]
        diff_min, diff_max = diff_smoothed.min(), diff_smoothed.max()
        diff_norm = (diff_smoothed - diff_min) / (diff_max - diff_min + 1e-8)

        # Improved thresholds (more spaced out)
        thresholds = [0.08, 0.22, 0.45]  # You can grid search/tune these

        # Initial multiclass mask (0 = no damage, 1 = minor, 2 = major, 3 = destroyed)
        mask = torch.zeros_like(diff_norm, dtype=torch.long)
        mask = torch.where((diff_norm >= thresholds[0]) & (diff_norm < thresholds[1]), 1, mask)  # Minor
        mask = torch.where((diff_norm >= thresholds[1]) & (diff_norm < thresholds[2]), 2, mask)  # Major
        mask = torch.where(diff_norm >= thresholds[2], 3, mask)  # Destroyed

    # Post-process: remove specks and smooth classes
    mask_np = mask.cpu().numpy()
    cleaned = clean_multiclass_mask(mask_np)

    return torch.from_numpy(cleaned).long()


# Metric calculation with weighted average for multiclass
def compute_mask_metrics(pred_mask, true_mask):
    pred = pred_mask.flatten().cpu().numpy().astype(int)
    true = true_mask.flatten().cpu().numpy().astype(int)
    return {
        'IoU': jaccard_score(true, pred, average='weighted'),
        'Dice': f1_score(true, pred, average='weighted'),
        'F1': f1_score(true, pred, average='weighted'),
        'Precision': precision_score(true, pred, average='weighted'),
        'Recall': recall_score(true, pred, average='weighted')
    }

# 95% Confidence Interval function
def compute_ci(arr):
    mean = np.mean(arr)
    std_err = np.std(arr) / np.sqrt(len(arr))
    return (mean - 1.96 * std_err, mean + 1.96 * std_err)

# Define categorical colormap for visualization of multiclass masks
damage_colors = ['black', 'blue', 'yellow', 'red']  # for classes 0 to 3
cmap = mcolors.ListedColormap(damage_colors)
norm = mcolors.BoundaryNorm(boundaries=[-0.5,0.5,1.5,2.5,3.5], ncolors=len(damage_colors))

# Get all pre-post pairs
low_income = ['congo', 'haiti']
pre_event_dir = r"C:\Users\sweta\Downloads\pre-event\pre-event_wo_ukraine_myanmar_mexico"
post_event_dir = r"C:\Users\sweta\Downloads\post-event\post-event"
all_pre_files = os.listdir(pre_event_dir)
filtered_pre_files = [f for f in all_pre_files if any(country.lower() in f.lower() for country in low_income)]
expected_post_files = [f.replace('_pre_disaster', '_post_disaster') for f in filtered_pre_files]
all_post_files = os.listdir(post_event_dir)
pairs = [
    (os.path.join(pre_event_dir, pre), os.path.join(post_event_dir, post))
    for pre, post in zip(filtered_pre_files, expected_post_files)
    if post in all_post_files
]

# Output directories setup - NEW multiclass folder inside lic_results
base_results_dir = r"C:\Users\sweta\anaconda_projects\non-trivial\evaluation_bias\lic_results"
output_dir = os.path.join(base_results_dir, "multiclass")
os.makedirs(output_dir, exist_ok=True)

disaster_output_dirs = {}
for d in disaster_types:
    dir_path = os.path.join(output_dir, d)
    os.makedirs(dir_path, exist_ok=True)
    disaster_output_dirs[d] = dir_path

# Evaluation loop
all_pair_metrics = []
disaster_type_metrics = {d: [] for d in disaster_types}

for pair_idx, (pre_path, post_path) in enumerate(pairs):
    print(f"\n=== Pair {pair_idx+1}/{len(pairs)} ===")
    pre_img_full = Image.open(pre_path).convert('RGB')
    post_img_full = Image.open(post_path).convert('RGB')

    tile_size = 256
    width, height = pre_img_full.size
    tiles = []

    for top in range(0, height, tile_size):
        for left in range(0, width, tile_size):
            box = (left, top, left + tile_size, top + tile_size)
            if box[2] <= width and box[3] <= height:
                pre_tile = pre_img_full.crop(box)
                post_tile = post_img_full.crop(box)
                tiles.append((pre_tile, post_tile))

    tile_metrics = []

    for idx, (pre_img, post_img) in enumerate(tiles):
        input_tensor = transform_rgb(pre_img).unsqueeze(0).to(device)
        post_resized = post_img.resize((256, 256))

        inferred_type = infer_disaster_type(pre_path)
        if inferred_type is None:
            print(f"Unknown disaster type for file: {pre_path}")
            continue

        disaster_idx = disaster_types.index(inferred_type)
        disaster_label = torch.tensor([[disaster_idx]], device=device)

        with torch.no_grad():
            fake_post, pred_mask = G(input_tensor, disaster_label)
        # Use multiclass mask function here
        actual_mask = SAR_damage_mask_multiclass_merged(pre_img, post_resized)
        fake_post_img = tensor_to_pil(fake_post)
        pred_mask = SAR_damage_mask_multiclass_merged(pre_img, fake_post_img)

        metrics = compute_mask_metrics(pred_mask, actual_mask)
        tile_metrics.append(metrics)

        # Save visualization every 10th tile
        if idx % 10 == 0:
            vis_dir = os.path.join(disaster_output_dirs[inferred_type], "visualizations")
            os.makedirs(vis_dir, exist_ok=True)

            fig, axs = plt.subplots(1, 4, figsize=(16, 4))

            axs[0].imshow(pre_img)
            axs[0].set_title("Pre-disaster")
            axs[0].axis('off')

            axs[1].imshow(post_img)
            axs[1].set_title("Post-disaster (True)")
            axs[1].axis('off')

            axs[2].imshow(fake_post_img)
            axs[2].set_title("Fake Post (Generated)")
            axs[2].axis('off')

            # Show predicted damage mask with categorical colors
            axs[3].imshow(pred_mask.cpu(), cmap=cmap, norm=norm)
            axs[3].set_title("Predicted Damage Mask")
            axs[3].axis('off')

            plt.suptitle(f"Pair {pair_idx+1}, Tile {idx} ({inferred_type})")
            save_path = os.path.join(vis_dir, f"pair{pair_idx+1}_tile{idx}.png")
            plt.savefig(save_path)
            plt.close(fig)

    mean_metrics = {
        'IoU': np.mean([m['IoU'] for m in tile_metrics]),
        'Dice': np.mean([m['Dice'] for m in tile_metrics]),
        'F1': np.mean([m['F1'] for m in tile_metrics]),
        'Precision': np.mean([m['Precision'] for m in tile_metrics]),
        'Recall': np.mean([m['Recall'] for m in tile_metrics]),
    }
    all_pair_metrics.append({
        'pair': (os.path.basename(pre_path), os.path.basename(post_path)),
        'metrics': mean_metrics
    })
    disaster_type_metrics[inferred_type].append(mean_metrics)

    print("Mean metrics for this pair:")
    for k, v in mean_metrics.items():
        print(f"  {k}: {v:.3f}")

# Overall stats
print("\n=== Overall Summary Across All Pairs ===")
all_iou = [m['metrics']['IoU'] for m in all_pair_metrics]
all_dice = [m['metrics']['Dice'] for m in all_pair_metrics]
all_f1 = [m['metrics']['F1'] for m in all_pair_metrics]
all_prec = [m['metrics']['Precision'] for m in all_pair_metrics]
all_rec = [m['metrics']['Recall'] for m in all_pair_metrics]

print(f"Average IoU: {np.mean(all_iou):.3f}")
print(f"Median IoU: {np.median(all_iou):.3f}")
print(f"Average Dice: {np.mean(all_dice):.3f}")
print(f"Median Dice: {np.median(all_dice):.3f}")
print(f"Average F1: {np.mean(all_f1):.3f}")
print(f"Median F1: {np.median(all_f1):.3f}")
print(f"Average Precision: {np.mean(all_prec):.3f}")
print(f"Median Precision: {np.median(all_prec):.3f}")
print(f"Average Recall: {np.mean(all_rec):.3f}")
print(f"Median Recall: {np.median(all_rec):.3f}")

# Save global results
with open(os.path.join(output_dir, "per_pair_metrics.json"), "w") as f:
    json.dump(all_pair_metrics, f, indent=2)

summary_stats = {
    'Average IoU': float(np.mean(all_iou)),
    'Median IoU': float(np.median(all_iou)),
    '95% CI IoU': list(compute_ci(all_iou)),

    'Average Dice': float(np.mean(all_dice)),
    'Median Dice': float(np.median(all_dice)),
    '95% CI Dice': list(compute_ci(all_dice)),

    'Average F1': float(np.mean(all_f1)),
    'Median F1': float(np.median(all_f1)),
    '95% CI F1': list(compute_ci(all_f1)),

    'Average Precision': float(np.mean(all_prec)),
    'Median Precision': float(np.median(all_prec)),
    '95% CI Precision': list(compute_ci(all_prec)),

    'Average Recall': float(np.mean(all_rec)),
    'Median Recall': float(np.median(all_rec)),
    '95% CI Recall': list(compute_ci(all_rec)),
}

with open(os.path.join(output_dir, "summary_stats.json"), "w") as f:
    json.dump(summary_stats, f, indent=2)

# Save results per disaster type
for d in disaster_types:
    metrics_list = disaster_type_metrics[d]
    if not metrics_list:
        continue  # skip if no data for this disaster

    iou_vals = [m['IoU'] for m in metrics_list]
    dice_vals = [m['Dice'] for m in metrics_list]
    f1_vals = [m['F1'] for m in metrics_list]
    prec_vals = [m['Precision'] for m in metrics_list]
    rec_vals = [m['Recall'] for m in metrics_list]

    summary = {
        'Average IoU': float(np.mean(iou_vals)),
        'Median IoU': float(np.median(iou_vals)),
        '95% CI IoU': list(compute_ci(iou_vals)),

        'Average Dice': float(np.mean(dice_vals)),
        'Median Dice': float(np.median(dice_vals)),
        '95% CI Dice': list(compute_ci(dice_vals)),

        'Average F1': float(np.mean(f1_vals)),
        'Median F1': float(np.median(f1_vals)),
        '95% CI F1': list(compute_ci(f1_vals)),

        'Average Precision': float(np.mean(prec_vals)),
        'Median Precision': float(np.median(prec_vals)),
        '95% CI Precision': list(compute_ci(prec_vals)),

        'Average Recall': float(np.mean(rec_vals)),
        'Median Recall': float(np.median(rec_vals)),
        '95% CI Recall': list(compute_ci(rec_vals)),
    }

    per_pair_path = os.path.join(disaster_output_dirs[d], f"per_pair_metrics_{d}.json")
    summary_path = os.path.join(disaster_output_dirs[d], f"summary_stats_{d}.json")

    disaster_pairs = [p for p in all_pair_metrics if infer_disaster_type(p['pair'][0]) == d]

    with open(per_pair_path, 'w') as f:
        json.dump(disaster_pairs, f, indent=2)
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)

    print(f"Saved results for disaster '{d}' to {disaster_output_dirs[d]}")

print(f"\nSaved all results to: {output_dir}")



=== Pair 1/196 ===


C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_clas

Mean metrics for this pair:
  IoU: 0.453
  Dice: 0.509
  F1: 0.509
  Precision: 0.881
  Recall: 0.511

=== Pair 2/196 ===


C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mean metrics for this pair:
  IoU: 0.421
  Dice: 0.575
  F1: 0.575
  Precision: 0.617
  Recall: 0.591

=== Pair 3/196 ===


...

C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_clas

Mean metrics for this pair:
  IoU: 0.443
  Dice: 0.513
  F1: 0.513
  Precision: 0.838
  Recall: 0.499

=== Pair 196/196 ===


C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_clas

Mean metrics for this pair:
  IoU: 0.275
  Dice: 0.326
  F1: 0.326
  Precision: 0.858
  Recall: 0.301

=== Overall Summary Across All Pairs ===
Average IoU: 0.403
Median IoU: 0.398
Average Dice: 0.538
Median Dice: 0.546
Average F1: 0.538
Median F1: 0.546
Average Precision: 0.653
Median Precision: 0.639
Average Recall: 0.537
Median Recall: 0.550
Saved results for disaster 'earthquake' to C:\Users\sweta\anaconda_projects\non-trivial\evaluation_bias\lic_results\multiclass\earthquake
Saved results for disaster 'volcano' to C:\Users\sweta\anaconda_projects\non-trivial\evaluation_bias\lic_results\multiclass\volcano

Saved all results to: C:\Users\sweta\anaconda_projects\non-trivial\evaluation_bias\lic_results\multiclass


C:\Users\sweta\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
